In [1]:
import otps
import pandas as pd

In [ ]:
name = 'nouakchott'
lat, lon = 17.99, -16.05

# # Calculate tide heights
observed_datetimes = pd.date_range(start='1987-01-01', end='2020-01-01', freq='30min').tolist()
observed_timepoints = [otps.TimePoint(lon, lat, dt) for dt in observed_datetimes]
observed_predictedtides = otps.predict_tide(observed_timepoints)
tideheights_m = [predictedtide.tide_m for predictedtide in observed_predictedtides]

# Create dataframe
tideheights_df = pd.DataFrame(index=observed_datetimes, data={'tide_height': tideheights_m})

# Write to file
tideheights_df.to_csv(f'{name}_{lat}_{lon}_tides.csv', index_label='time')

In [ ]:
## Test tide points

In [2]:
import geopandas as gpd

In [ ]:
tide_points_gpd = gpd.read_file('/g/data/r78/rt1527/dea-notebooks/Waterline_extraction/raw_data/tide_points_coastal.shp').to_crs({'init': 'EPSG:4326'})

In [ ]:
x_vals = tide_points_gpd.geometry.centroid.x
y_vals = tide_points_gpd.geometry.centroid.y
observed_datetimes = pd.date_range(start='2000-01-01', end='2000-01-01', freq='14D').tolist()
observed_timepoints = [otps.TimePoint(lon, lat, observed_datetimes[0]) for lon, lat in zip(x_vals, y_vals)]
observed_predictedtides = otps.predict_tide(observed_timepoints)

In [ ]:
points_df = pd.DataFrame([(i.timepoint.lon, i.timepoint.lat, i.tide_m) for i in observed_predictedtides], columns=['lon', 'lat', 'tide_m']) 
points_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points_df.lon, points_df.lat), crs={'init': 'EPSG:4326'})
points_gdf.to_file('/g/data/r78/rt1527/dea-notebooks/Waterline_extraction/raw_data/tide_points_coastal2.shp')

In [3]:
import datacube

query = {'x': (122.12, 122.41),
         'y': (-18.24, -17.88),
         'time': (2015, 2017)}

dc = datacube.Datacube(app='pixel_drill')
landsat_ds = dc.load(product='ls8_nbart_albers', **query, dask_chunks={'time': 1})


In [4]:
import shapely.wkt
bounds = shapely.wkt.loads(landsat_ds.geobox.geographic_extent.buffer(0.1).wkt)

In [5]:
points_gdf = gpd.read_file('/g/data/r78/rt1527/dea-notebooks/Waterline_extraction/raw_data/tide_points_coastal2.shp')
subset_gdf = points_gdf[points_gdf.geometry.intersects(bounds)]

In [6]:
x_vals = subset_gdf.geometry.centroid.x
y_vals = subset_gdf.geometry.centroid.y
observed_datetimes = landsat_ds.time.data.astype('M8[s]').astype('O').tolist()

# Create list of lat/lon/time scenarios to model
observed_timepoints = [otps.TimePoint(lon, lat, date) 
                       for date in observed_datetimes
                       for lon, lat in zip(x_vals, y_vals)]

# Model tides for each scenario
observed_predictedtides = otps.predict_tide(observed_timepoints)

In [7]:
import numpy as np
points_df = pd.DataFrame([(i.timepoint.timestamp, i.timepoint.lon, i.timepoint.lat, i.tide_m) 
                          for i in observed_predictedtides], columns=['time', 'lon', 'lat', 'tide_m']) 

points_gdf = gpd.GeoDataFrame(data={'i': np.unique(points_df.time, return_inverse=True)[1], 
                                    'tide_m': points_df.tide_m}, 
                                 geometry=gpd.points_from_xy(points_df.lon, points_df.lat), 
                                 crs={'init': 'EPSG:4326'}).to_crs({'init': 'EPSG:3577'})

In [2]:
import numpy as np
import scipy.interpolate
from scipy import ndimage as nd

def interp_tides(ds, x_coords, y_coords, z_coords, 
                 method='linear', fill_nearest=False, sigma=None):
    
    # Extract xy and elev points
    points_xy = np.vstack([x_coords, y_coords]).T

    # Create grid to interpolate into
    grid_y, grid_x = np.meshgrid(ds.x, ds.y)  

    # Interpolate x, y and z values using linear/TIN interpolation
    out = scipy.interpolate.griddata(points=points_xy, 
                                     values=z_coords, 
                                     xi=(grid_y, grid_x), 
                                     method=method)

    # Calculate nearest
    if fill_nearest:
        
        nearest_inds = nd.distance_transform_edt(input=np.isnan(out), 
                                                 return_distances=False, 
                                                 return_indices=True)
        out = out[tuple(nearest_inds)]
        
    # Apply guassian filter        
    if sigma:

        out = nd.filters.gaussian_filter(out, sigma=sigma)
        
    return out


# Extract vertex coordinates and heights from geopandas
def contours_to_arrays(gdf, col):

    coords_zvals = []

    for i in range(0, len(gdf)):

        val = gdf.iloc[i][col]

        try:
            coords = np.concatenate([np.vstack(x.coords.xy).T for x in gdf.iloc[i].geometry])

        except:
            coords = np.vstack(gdf.iloc[i].geometry.coords.xy).T

        coords_zvals.append(np.column_stack((coords, np.full(np.shape(coords)[0], fill_value=val))))

    return np.concatenate(coords_zvals)



In [9]:
out = []

for i in range(0, len(landsat_ds.time)):
    
    x_coords = points_gdf[points_gdf.i == i].geometry.x, 
    y_coords = points_gdf[points_gdf.i == i].geometry.y, 
    z_coords = points_gdf[points_gdf.i == i].tide_m

    out_tide = interp_tides(ds=landsat_ds.isel(time=i), 
                            x_coords=x_coords, 
                            y_coords=y_coords, 
                            z_coords=z_coords, sigma=50)
    
    out.append(out_tide)


In [10]:
# out
import xarray as xr
test = xr.DataArray(np.dstack(out), coords=[landsat_ds.y, landsat_ds.x, landsat_ds.time], dims=['y', 'x', 'time']) 


In [ ]:
# points_df.time[0].isoformat()

In [21]:
import gc
gc.collect()

0

In [ ]:
landsat_ds = landsat_ds.where(test > test.median(dim='time'))

In [ ]:
out = None